# 🔧 Simulation 3D du champ de pression acoustique (StructuredGrid)

In [ ]:

# 🧩 Installation si nécessaire
try:
    import pyvista as pv
except ImportError:
    !pip install -q pyvista
    import pyvista as pv

import numpy as np
pv.set_jupyter_backend("static")


In [ ]:

# ⚙️ Paramètres
N = 60
L = 200  # mm
X = np.linspace(0, L, N)
Y = np.linspace(0, L, N)
Z = np.linspace(0, L, N)
AMPL = 5.0
WAVE_SPEED = 343000.0
FREQ = 40000.0
K = 2 * np.pi * FREQ / WAVE_SPEED

# 🎯 Point de focalisation
fx, fy, fz = L / 2, L / 2, L / 2

# 🔊 Transducteurs (5×5 haut et bas)
transducers = []
for i in range(5):
    for j in range(5):
        x = fx + (i - 2) * 20
        y = fy + (j - 2) * 20
        for z in [0, L]:
            r = np.sqrt((x - fx)**2 + (y - fy)**2 + (z - fz)**2)
            phase = -K * r
            transducers.append((x, y, z, phase))


In [ ]:

# 🧮 Calcul du champ de pression
volume = np.zeros((N, N, N), dtype=np.float32)
for ix, x in enumerate(X):
    for iy, y in enumerate(Y):
        for iz, z in enumerate(Z):
            p = 0
            for tx, ty, tz, ph in transducers:
                r = np.sqrt((x - tx)**2 + (y - ty)**2 + (z - tz)**2) + 0.1
                p += AMPL * np.sin(K * r + ph) / r
            volume[ix, iy, iz] = abs(p)
volume /= np.max(volume)


In [ ]:

# 🧊 Création du StructuredGrid
from pyvista import StructuredGrid

xx, yy, zz = np.meshgrid(X, Y, Z, indexing="ij")
grid = StructuredGrid(xx, yy, zz)
grid["pressure"] = volume.flatten(order="F")


In [ ]:

# 🎨 Visualisation avec transparence et couleur
plotter = pv.Plotter()
opacity = [0.0, 0.01, 0.05, 0.1, 0.2, 0.4, 0.6, 0.9, 1.0]
plotter.add_volume(grid, scalars="pressure", cmap="coolwarm", opacity=opacity, shade=True)
plotter.add_axes()
plotter.show_grid()
plotter.show()
